# Base-line

### Тетрадь с предварительной обработкой новостей и определение тональности новостей

#### Тональность новости обозначается sentiment score и может принимать значения в диапазоне [1 - 5], где

* 1: что-то очень негативное относительно компании или дана рекомендация "продавать"

* 2: что-то скорее негативное, например, вышла отчетность ниже ожиданий, проблемы на каком-нибудь заводе, санкции и пр.

* 3: Нейтральная новость. Важно! Если текст

> "Акция выросла на 40% за день"

Считается нейтральной новостью (т.к. цены акций - скорее [мартингал](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%B3%D0%B0%D0%BB)).

> "Акция выросла на 40% за день, потому что ...

Считается положительной новостью (sentimen\_score > 3), т.к. есть объяснение

* 4: Что-то положительное, например, вышла отчетность выше ожиданий, успехи на каком-нибудь заводе, новый контракт и пр.

* 5: Что-то очень положительное или есть рекомендация "покупать" или "входит в подборку нашух супер-акций"

## Импорт библиотек

In [16]:
import numpy as np
import pandas as pd
from time import time
from tqdm import notebook
import os
import re

In [370]:
from collections import Counter

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [108]:
import emoji
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk import word_tokenize
import string

In [115]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iakulov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Чтение фалов

In [10]:
NOTEBOOK_PATH = os.getcwd()
SENTIMENT_PATH = NOTEBOOK_PATH + '\\sentiment_dataset' 

In [20]:
data_combined_hackathon = pd.read_csv(f'{SENTIMENT_PATH}\\data_combined_hackathon.csv')
data = data_combined_hackathon[data_combined_hackathon['MessageText']!=''] 
data = data.loc[~data.MessageText.isna()]
data['uniq_id_message'] = data.ChannelID.astype(str) + data.messageid.astype(str)

In [21]:
data.head(1)

,ChannelID,messageid,issuerid,MessageText,EMITENT_FULL_NAME,BGTicker,OtherTicker,SentimentScore,uniq_id_message
0,1239405989,4486,115,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,"публичное акционерное общество ""НОВАТЭК""",NVTK RX,NaN,3.0,12394059894486


In [330]:
df_ticker_id = pd.read_csv(f'{SENTIMENT_PATH}\\company_name_id.csv')

## Определение компании из новости

Определение происходит по алгоритму: 
 - Замены всех смайликов, знаков перечисления ('•') на '.'. 
 - Разделение большой новости по предложениям
 - Разделение предложния на слова и на их основе создание множества слов - X
 - Поиск слов, которые попали в пересечение множества X и множества Y, где Y - это множество названий компаний из файла company_name_id.csv
 - Соотношение предложения с компанией и id компании

In [338]:
_tmp_data = data[['MessageText', 'uniq_id_message']].drop_duplicates()
input_list_example = _tmp_data.MessageText.values.tolist()
input_list_id_example = _tmp_data.uniq_id_message.values.tolist()
del _tmp_data

In [168]:
punctuation = string.punctuation + "•«»``''"

In [169]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~•«»``\'\''

In [339]:
start = time()
dict_info_example = {
    'uniq_id_message':[],
    'news':[],
    'tiker': [],
    'issuerid':[],
}
all_list_ticker_issure = df_ticker_id.name.values.tolist()
for i, news, messageid in zip(notebook.tqdm(range(len(input_list_example))),input_list_example, input_list_id_example):
    news = emoji.replace_emoji(news, replace=' . ')
    news = re.sub(r'http\S+', '', news)
    news = re.sub(r'.ru\S+', '', news)
    
    all_info_news = [word for word in word_tokenize(news) if (word not in punctuation) and (word not in stopwords)]
    list_to_set = all_info_news+ list(map(lambda x: x.upper(), all_info_news)) + list(map(lambda x: x[:-1], all_info_news))
#     print(all_info_news)
    for union_tickers in list(set(list_to_set) & set(all_list_ticker_issure)):
#         print(union_tickers)
        for index_word, word in enumerate(all_info_news):
            if (union_tickers.upper() in word.upper()) or (union_tickers.upper() in word.upper()[:-1]):
                if index_word < 10:
                    news_str = all_info_news[:index_word+10]
                elif index_word == len(all_info_news):
                    news_str = all_info_news[index_word-10:]
                else:
                    news_str = all_info_news[index_word-5:index_word+5]
                issuerid = df_ticker_id.loc[df_ticker_id.name == union_tickers]['issuerid'].values.tolist()[0]
                dict_info_example['news'].append(' '.join(news_str))
                dict_info_example['tiker'].append(union_tickers)
                dict_info_example['uniq_id_message'].append(messageid)
                dict_info_example['issuerid'].append(issuerid)
# print(dict_info_example)
df_markup_news = pd.DataFrame.from_dict(dict_info_example).drop_duplicates().reset_index(drop=True)

  0%|          | 0/7255 [00:00<?, ?it/s]

In [340]:
len(input_list_example)

7255

In [341]:
len(df_markup_news.uniq_id_message.unique())

7062

In [342]:
df_news_sent = data[['uniq_id_message', 'SentimentScore']].rename(columns={'SentimentScore': 'sentiment'}).merge(df_markup_news,
                                                                                                 how='inner', 
                                                                                                 on='uniq_id_message').drop_duplicates().reset_index(drop=True).copy()

In [343]:
df_news_sent.head(2)

,uniq_id_message,sentiment,news,tiker,issuerid
0,12394059894486,3.0,ограничить экспорт технологий КНР Bloomberg НО...,НОВАТЭ,115
1,12394059894486,3.0,объявила банкротстве вслед FTX Bloomberg NVTK,NVTK,115


In [345]:
df_news_sent = df_news_sent.drop_duplicates(['uniq_id_message', 'sentiment', 'news', 'issuerid'])

In [348]:
df_join_news = df_news_sent.groupby(['uniq_id_message', 'issuerid'], as_index=False).apply(lambda x: ' '.join(x['news'].values.tolist())).copy()
df_join_news = df_join_news.rename(columns={None:'join_news'})

In [390]:
df_join_news_sent = df_join_news.merge(df_news_sent[['uniq_id_message', 
                                 'sentiment', 
                                 'tiker', 
                                 'issuerid']], 
                   how='inner', 
                   on=['uniq_id_message','issuerid']).drop_duplicates().reset_index(drop=True).copy()

In [391]:
df_join_news_sent = df_join_news_sent.reset_index()

In [392]:
df_join_news_sent['join_news'] = df_join_news_sent['join_news'].apply(lambda x: " ".join(Counter(x.split(' ')).keys()) )

## Обучение модели

In [273]:
def calculate_metriks(pred, target) -> dict:
    accuracy = accuracy_score(pred, target)
    f1 = f1_score(pred, target, average='macro')
    team_score = 100 * (0.5*f1 + 0.5*accuracy)
    
    return {'accuracy':accuracy,
           'f1':f1,
           'team_score':team_score}

### Векторизация

In [39]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060]
[nltk_data]     Попытка установить соединение была безуспешной, т.к.
[nltk_data]     от другого компьютера за требуемое время не получен
[nltk_data]     нужный отклик, или было разорвано уже установленное
[nltk_data]     соединение из-за неверного отклика уже подключенного
[nltk_data]     компьютера>


In [393]:
df_train = df_join_news_sent.sample(frac=0.8).copy()
df_test = df_join_news_sent.loc[~df_join_news_sent['index'].isin(df_train['index'].values.tolist())]

In [394]:
corpus = df_train['join_news'].values.astype('U')

In [395]:
X, y = corpus, df_train.sentiment

In [396]:
max_ind = 0
max_len = 0
max_str = ''

for ind, i in enumerate(X):
    if len(i) > max_len:
        max_len = len(i)
        max_ind = ind
        max_str = i

In [397]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [398]:
tf_idf = count_tf_idf.fit_transform(X)

In [399]:
tf_idf.shape

(20798, 19808)

In [404]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf, y, test_size=0.2)

### Обучение логистической регресии с перебором гиперпараметров

In [405]:
param_grid = {'C': np.arange(1e-03, 2, 0.01)} # hyper-parameter list to fine-tune
log_gs = GridSearchCV(LogisticRegression(solver='liblinear', # setting GridSearchCV
                                         class_weight="balanced", 
                                         random_state=7),
                      return_train_score=True,
                      param_grid=param_grid,
                      scoring='accuracy',
                      cv=10)

In [406]:
log_grid = log_gs.fit(X_train, y_train)

In [407]:
log_opt = log_grid.best_estimator_
results = log_gs.cv_results_

In [408]:
log_opt.fit(X_train, y_train)

LogisticRegression(C=1.9909999999999997, class_weight='balanced',
                   random_state=7, solver='liblinear')

In [409]:
log_opt.score(X_train, y_train)

0.7638538285851665

In [410]:
res_test_df = log_opt.predict(X_test) #log_reg.predict(X_test)

In [411]:
calculate_metriks(pred=res_test_df, target=y_test)

{'accuracy': 0.5875, 'f1': 0.5093663130477694, 'team_score': 54.84331565238847}

## Тест модели

In [412]:
corpus_for_test = df_test['join_news'].values.astype('U')

In [413]:
X_for_test = count_tf_idf.transform(corpus_for_test)

In [414]:
test_pred = log_opt.predict(X_for_test)

In [419]:
test_target = df_test.sentiment.values.tolist()

In [420]:
calculate_metriks(pred=test_pred, target=test_target)

{'accuracy': 0.5905769230769231,
 'f1': 0.5069623608869254,
 'team_score': 54.876964198192425}